<a href="https://colab.research.google.com/github/suhyeon0325/TIL/blob/main/laidd/extract_active.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdkit-pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 32.8 MB/s eta 0:00:00


In [2]:
import os
import pandas as pd
from rdkit import Chem

In [3]:
df = pd.read_csv("/content/drive/MyDrive/파이널/data/curated-solubility-dataset.csv")

In [5]:
df.head()

,ID,Name,InChI,InChIKey,SMILES,Solubility,SD,Ocurrences,Group,MolWt,...,NumRotatableBonds,NumValenceElectrons,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT
0,A-3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1,G1,392.510,...,17.0,142.0,0.0,0.0,0.0,0.0,0.00,158.520601,0.000000e+00,210.377334
1,A-4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1,G1,169.183,...,0.0,62.0,2.0,0.0,1.0,3.0,29.10,75.183563,2.582996e+00,511.229248
2,A-5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1,G1,140.569,...,1.0,46.0,1.0,0.0,0.0,1.0,17.07,58.261134,3.009782e+00,202.661065
3,A-8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.0,1,G1,756.226,...,10.0,264.0,6.0,0.0,0.0,6.0,120.72,323.755434,2.322963e-07,1964.648666
4,A-9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1,G1,422.525,...,12.0,164.0,2.0,4.0,4.0,6.0,56.60,183.183268,1.084427e+00,769.899934


In [7]:
# 주성분 코드, 활성 성분, 염(salt)을 추출하는 과정
list_to_remove = []
for index, row in df.iterrows():
    smi = row['SMILES']  # SMILES 코드를 가져옴
    mol = Chem.MolFromSmiles(smi)  # SMILES 코드를 분자 구조로 변환

    if '.' in smi:  # SMILES 코드에 '.'이 포함되어 있는지 확인 (이는 여러 조각으로 구성된 분자임을 의미)
        try:
            # 분자를 조각(fragments)으로 나누고 각각의 조각을 분자 리스트로 변환
            fragments_list = list(Chem.GetMolFrags(mol, asMols=True))
            # 각 조각의 원자 수를 계산
            fragments_sizes = [frag.GetNumAtoms() for frag in fragments_list]

            # 가장 큰 조각을 찾음
            largest_i = fragments_sizes.index(max(fragments_sizes))
            largest_frag = fragments_list[largest_i]

            # 가장 큰 조각의 SMILES 코드 (활성 성분)
            active = Chem.MolToSmiles(largest_frag)

            # 가장 큰 조각을 리스트에서 제거하고 남은 조각들을 염(salt)으로 취급
            fragments_list.remove(largest_frag)
            salt_smi_list = [Chem.MolToSmiles(frag) for frag in fragments_list]
            salt = '.'.join(salt_smi_list)

            # 데이터프레임에 활성 성분과 염(salt) 정보를 추가
            df.loc[index, 'active ingredient'] = active
            df.loc[index, 'salt'] = salt

        except:
            # 예외 발생 시 인덱스와 SMILES 코드를 출력하고 리스트에 인덱스를 추가
            print(index, smi, str(e))
            list_to_remove.append(index)

[06:56:37] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not removing hydrogen atom without neighbors
[06:56:38] WARNING: not r

In [8]:
# 리스트에 있는 인덱스에 해당하는 행을 제거 (예외가 발생한 경우)
df = df.drop(list_to_remove)

In [9]:
df.head()

,ID,Name,InChI,InChIKey,SMILES,Solubility,SD,Ocurrences,Group,MolWt,...,NumAromaticRings,NumSaturatedRings,NumAliphaticRings,RingCount,TPSA,LabuteASA,BalabanJ,BertzCT,active ingredient,salt
0,A-3,"N,N,N-trimethyloctadecan-1-aminium bromide",InChI=1S/C21H46N.BrH/c1-5-6-7-8-9-10-11-12-13-...,SZEMGTQCPRNXEG-UHFFFAOYSA-M,[Br-].CCCCCCCCCCCCCCCCCC[N+](C)(C)C,-3.616127,0.0,1,G1,392.510,...,0.0,0.0,0.0,0.0,0.00,158.520601,0.000000e+00,210.377334,CCCCCCCCCCCCCCCCCC[N+](C)(C)C,[Br-]
1,A-4,Benzo[cd]indol-2(1H)-one,InChI=1S/C11H7NO/c13-11-8-5-1-3-7-4-2-6-9(12-1...,GPYLCFQEKPUWLD-UHFFFAOYSA-N,O=C1Nc2cccc3cccc1c23,-3.254767,0.0,1,G1,169.183,...,2.0,0.0,1.0,3.0,29.10,75.183563,2.582996e+00,511.229248,NaN,NaN
2,A-5,4-chlorobenzaldehyde,InChI=1S/C7H5ClO/c8-7-3-1-6(5-9)2-4-7/h1-5H,AVPYQKSLYISFPO-UHFFFAOYSA-N,Clc1ccc(C=O)cc1,-2.177078,0.0,1,G1,140.569,...,1.0,0.0,0.0,1.0,17.07,58.261134,3.009782e+00,202.661065,NaN,NaN
3,A-8,"zinc bis[2-hydroxy-3,5-bis(1-phenylethyl)benzo...",InChI=1S/2C23H22O3.Zn/c2*1-15(17-9-5-3-6-10-17...,XTUPUYCJWKHGSW-UHFFFAOYSA-L,[Zn++].CC(c1ccccc1)c2cc(C(C)c3ccccc3)c(O)c(c2)...,-3.924409,0.0,1,G1,756.226,...,6.0,0.0,0.0,6.0,120.72,323.755434,2.322963e-07,1964.648666,CC(c1ccccc1)c1cc(C(=O)[O-])c(O)c(C(C)c2ccccc2)c1,[Zn+2].CC(c1ccccc1)c1cc(C(=O)[O-])c(O)c(C(C)c2...
4,A-9,4-({4-[bis(oxiran-2-ylmethyl)amino]phenyl}meth...,InChI=1S/C25H30N2O4/c1-5-20(26(10-22-14-28-22)...,FAUAZXVRLVIARB-UHFFFAOYSA-N,C1OC1CN(CC2CO2)c3ccc(Cc4ccc(cc4)N(CC5CO5)CC6CO...,-4.662065,0.0,1,G1,422.525,...,2.0,4.0,4.0,6.0,56.60,183.183268,1.084427e+00,769.899934,NaN,NaN


In [10]:
df.to_csv("/content/drive/MyDrive/파이널/data/aqsol_actives.csv", index=False)